Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
!git clone https://github.com/152334H/tortoise-tts-fast
%cd tortoise-tts-fast
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .
!pip3 install git+https://github.com/152334H/BigVGAN.git

## **Restart the runtime!**

In [1]:
# @title # Setup
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

/home/ansel/.pyenv/versions/3.10.15/envs/fast/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ansel/.pyenv/versions/3.10.15/envs/fast/lib/python3.10/site-packages/transformers/modeling_utils.py:446: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u

In [2]:
# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"  # @param {type:"string"}
# @markdown Show code for multiline text input
# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"  # @param ["ultra_fast", "fast", "standard", "high_quality"]

In [3]:
#@markdown Tortoise will attempt to mimic voices you provide. It comes pre-packaged
#@markdown with some voices you might recognize.

#@markdown Let's list all the voices available. These are just some random clips I've gathered
#@markdown from the internet as well as a few voices from the training dataset.
#@markdown Feel free to add your own clips to the voices/ folder.
%cd tortoise-tts-fast
%ls tortoise/voices
import IPython
IPython.display.Audio(filename='tortoise/voices/tom/1.wav')

[Errno 2] No such file or directory: 'tortoise-tts-fast'
/home/ansel/dev/tortoise-tts-fast
angie/                freeman/  myself/        tom/            train_grace/
applejack/            geralt/   pat/           train_atkins/   train_kennard/
cond_latent_example/  halle/    pat2/          train_daws/     train_lescault/
daniel/               jlaw/     rainbow/       train_dotrice/  train_mouse/
deniro/               lj/       snakes/        train_dreams/   weaver/
emma/                 mol/      tim_reynolds/  train_empire/   william/


/home/ansel/.pyenv/versions/3.10.15/envs/fast/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [4]:
# @markdown Pick one of the voices from the output above
voice = "train_dotrice"  # @param {type:"string"}

# @markdown Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(
    text,
    voice_samples=voice_samples,
    conditioning_latents=conditioning_latents,
    preset=preset,
)
torchaudio.save("generated.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio("generated.wav")

mode 0


/home/ansel/dev/tortoise-tts-fast/tortoise/models/arch_util.py:354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch.load(self.mel_norm_file)


Generating autoregressive samples..


100%|██████████| 6/6 [00:09<00:00,  1.58s/it]


Computing best candidates using CLVP


100%|██████████| 6/6 [00:00<00:00, 13.35it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 20/20 [00:00<00:00, 49.84it/s]


In [ ]:
# @markdown Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# @markdown (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(
    text, voice_samples=None, conditioning_latents=None, preset=preset
)
torchaudio.save("generated.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio("generated.wav")

In [ ]:
# @markdown Optionally, upload use your own voice by running the next two cells. I recommend
# @markdown you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "custom"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
    with open(os.path.join(custom_voice_folder, f"{i}.wav"), "wb") as f:
        f.write(file_data)

In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(
    text,
    voice_samples=voice_samples,
    conditioning_latents=conditioning_latents,
    preset=preset,
)
torchaudio.save(f"generated-{CUSTOM_VOICE_NAME}.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f"generated-{CUSTOM_VOICE_NAME}.wav")

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(["pat", "william"])

gen = tts.tts_with_preset(
    "They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.",
    voice_samples=voice_samples,
    conditioning_latents=conditioning_latents,
    preset=preset,
)
torchaudio.save("captain_kirkard.wav", gen.squeeze(0).cpu(), 24000)
IPython.display.Audio("captain_kirkard.wav")